In [1]:
from SMT_model import *
from time import time as time_clock

In [6]:
#Demo with instance 01
Path = ('../Instances/inst07.dat')

timeout = 300


In [7]:
#Read the file from the path
with open(Path, 'r') as file:
    data = file.read()

instance = data_parsing(data)
print(instance)

{'m': 6, 'n': 17, 'l': [190, 185, 185, 190, 195, 185], 's': [11, 11, 23, 16, 2, 1, 24, 14, 20, 10, 11, 22, 2, 12, 9, 21, 10], 'D': [[0, 20, 19, 28, 58, 48, 45, 32, 90, 61, 71, 59, 65, 46, 72, 51, 46, 66], [103, 0, 81, 107, 38, 110, 55, 94, 76, 123, 88, 76, 69, 86, 99, 113, 108, 106], [73, 41, 0, 80, 40, 29, 26, 13, 75, 42, 70, 58, 46, 27, 53, 32, 27, 47], [28, 48, 47, 0, 55, 59, 52, 60, 62, 58, 43, 31, 40, 21, 59, 48, 53, 61], [83, 38, 79, 87, 0, 102, 58, 92, 56, 103, 68, 56, 49, 67, 79, 94, 106, 86], [76, 44, 72, 83, 43, 0, 98, 16, 46, 64, 73, 61, 73, 67, 27, 40, 30, 43], [65, 55, 26, 64, 58, 55, 0, 39, 69, 68, 50, 38, 26, 31, 66, 58, 53, 56], [60, 28, 56, 67, 27, 16, 82, 0, 62, 48, 75, 63, 69, 51, 40, 24, 14, 34], [58, 76, 77, 81, 55, 46, 69, 62, 0, 56, 43, 31, 43, 53, 23, 80, 76, 30], [74, 79, 42, 84, 71, 35, 68, 51, 56, 0, 59, 47, 54, 37, 59, 24, 65, 43], [15, 35, 34, 43, 12, 43, 41, 47, 39, 47, 0, 12, 24, 10, 16, 37, 61, 32], [27, 47, 46, 50, 24, 55, 38, 59, 31, 47, 12, 0, 12, 22,

In [8]:
def run_smt(_, instance, timeout, sb=False):
    generation_start_time = time_clock()

    o, x, max_distance = stm_model(instance, timeout, sb)
    generation_duration = time_clock() - generation_start_time
    o.set("timeout", int(timeout - generation_duration) * 1000)

    obj = o.minimize(max_distance)
    res = o.check()
    print(res)
    if res in [sat, unknown]:
        try:
            result_formatted = format_solution(instance, o.model(), x)
            return result_formatted, res==sat, obj.value()
        except Exception as e:
            return e, obj.value(), o.reason_unknown()
    elif res == unsat:
        return "unsat"
    else:
        return "unknown"

In [9]:
print(run_smt(_, instance, timeout, False))

unknown
[x_1_4 = 17,
 x_5_17 = 17,
 x_0_6 = 17,
 x_3_1 = 2,
 x_0_8 = 12,
 x_3_14 = 17,
 x_0_4 = 17,
 x_3_11 = 17,
 x_2_4 = 17,
 x_3_2 = 17,
 x_3_16 = 17,
 x_3_3 = 17,
 x_0_1 = 17,
 x_4_6 = 17,
 x_1_15 = 17,
 x_0_12 = 17,
 x_1_13 = 17,
 x_2_1 = 17,
 x_1_17 = 17,
 x_1_10 = 17,
 x_5_14 = 17,
 x_5_15 = 17,
 x_0_11 = 17,
 x_2_15 = 17,
 x_4_7 = 17,
 x_5_7 = 17,
 x_3_13 = 17,
 x_3_5 = 17,
 x_1_11 = 17,
 x_2_7 = 17,
 x_2_5 = 17,
 x_0_7 = 17,
 x_0_16 = 17,
 x_5_1 = 17,
 x_0_15 = 17,
 s = Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(K(Int,
                                        11),
                                        7,
                                        14),
                                        6,
                                        24),
                                        14,
                                        9),
                                        11,
                                        22),
                     